# ```aggregate```: Simple Examples

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from IPython.core.display import HTML, display
from importlib import reload

# pandas options 
pd.set_option('max_rows', 50)
pd.set_option('max_columns', 30)
pd.set_option('display.max_colwidth', 150)

# matplotlib and plotting options 
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# seaborn options 
sns.set(context='paper', style='darkgrid', font='serif')
# sns.set(context='paper', style='ticks', font='serif')

# this file is in examples
import sys
sys.path.insert(0,'/s/telos/python/aggregate_project/')
import aggregate as agg

In [ ]:
uw = agg.Underwriter()

In [ ]:
uw.list()

In [ ]:
uw.describe(pretty_print=True);

In [ ]:
uw.describe('severity')

In [ ]:
uw['liaba']

In [ ]:
uw.liabc.plot()

In [ ]:
ag = uw.ppal
ag.easy_update(14)
ag.report('audit')

In [ ]:
pf = uw.bodoff1
pf.update(8, 1)

In [ ]:
pf.plot('density')

In [ ]:
c = uw.write('a: 0.01 * uw.cmp', 'test', update=True, log2=13) 
c.plot(subplots=True, height=4)

In [ ]:
biz = uw['homeowners'] 
print(f'Type: {type(biz)}\nstr:  {biz}\nrepr: {repr(biz)}')

In [ ]:
biz = uw.homeowners 
print(f'Type: {type(biz)}\nstr:  {biz}\nrepr: {repr(biz)}')

In [ ]:
biz.easy_update(10)
print(f'Type: {type(biz)}\nstr:  {biz}\nrepr: {repr(biz)}')
biz.report('audit')

In [ ]:
biz.recommend_bucket(verbose=True)

In [ ]:
a, b = biz.easy_update(10, reporting_level=1, verbose=True)
display(a)
display(b)

In [ ]:
biz.audit_df

In [ ]:
biz.report('all')

# Script Examples

In [ ]:
s = uw.write('sev MyLN1 12 * lognorm 1; sev MyLN2 12 * lognorm 2; sev MyLN3 12 * lognorm 3; ')

In [ ]:
for v in s:
    print(v.moms())
    v.plot()
print([ 12 * np.exp(x*x/2) for  x in [1,2,3]])

In [ ]:
pf = uw.write('PA: 0.0085 * uw.pers_auto; CA: 0.02 * uw.comm_auto;WC: 0.005 * uw.work_comp', 'test', True, False, log2=16, bs=10e4, remove_fuzz=True)

In [ ]:
print(pf)

In [ ]:
pf.report('quick')

In [ ]:
pf.plot('audit', aspect=1.4, height=2.25)

## More complex program 

In [ ]:
ans = uw.write("""
A1: 50  claims          on gamma 12 cv .30 (mixed gamma 0.014)
A2: 50  claims 30 xs 10 on gamma 12 cv .30 (mixed gamma 0.014)
A3: 50  claims          on gamma 12 cv 1.30 (mixed gamma 0.014)
A4: 50  claims 30 xs 20 on gamma 12 cv 1.30 (mixed gamma 0.14); B 15 claims 15 xs 15 on lognorm 12 cv 1.5 + 2 mixed gamma 4.8
Cat 1.7 claims 25 xs 5  on 25 * pareto 1.3 0 - 25 poisson 
ppa: 1e-8 * uw.ppal
""", 'my_first_program', True, False, log2=13, bs=0.25, remove_fuzz=True, trim_df=False)

In [ ]:
ans.recommend_bucket()

In [ ]:
ans.update(13, 1, remove_fuzz=True)

In [ ]:
ans.plot('density', subplots=True, logy=True)

In [ ]:
ans.report('audit')

In [ ]:
for a in ans:
    print(a.name, a.attachment, a.limit, a.recommend_bucket(), a.agg_m, a.agg_cv, a.agg_skew)

In [ ]:
for a in ans:
    print(a)

In [ ]:
ans.report()

In [ ]:
for col in ['Cat']:
    print(col)
    t1 = ans.density_df.loc[:, 'e_' + col]
    t2 = ans.density_df.loc[:, 'exa_' + col]
    print(t2[np.isnan(t2)] )

In [ ]:
type(t2)

In [ ]:
n = np.argwhere(np.isnan(t2))
ans.density_df.filter(regex='p_total|ex(le)?a_A$|loss|F|S|exi_xgta_A$').iloc[n[0][0]-10:n[0][0]+10, :]

In [ ]:
t2[1500:2000]

# Integrated Parser

In [ ]:
program1 = """
A:    50  claims,             on  gamma 12 cv .30          mixed gamma 0.014
Ba:  500 loss,                on lognorm 50 cv .8
Bb:  500 loss,  1000 xs 0     on lognorm 50 cv .8
Bg:  500 loss,                on gamma 50 cv .8
C:   500  loss,      75 xs 25, on lognorm 50 cv .9
D:    25  claims,    30 xs 20,  on gamma 12 cv 1.30         (mixed gamma 0.85)
Cat1:  1.7 claims,  125 xs 5,   on 25 * pareto 1.3 - 25 
Cat2:  3.5 claims, 1000 xs 0,  on 25 * pareto 2.3 0 - 25
"""
program2 = """
Thick:  500 loss,            on lognorm 50 cv .8
Thin:  500 loss,  1000 xs 0  on lognorm 50 cv .8
Cat:  2 claims,  1250 xs 5,  on 25 * pareto 1.3 - 25 
"""

program3 = '''
MyWC: 0.005 * uw.work_comp
InHomCA: uw.comm_auto * 0.002 ;
HomCA: 0.001 * uw.comm_auto
Cat: 50000000 loss 1e9 xs 0 on 50000000 * pareto 1.3 - 50000000
'''

ans1 = uw.write(program1, 'script example 1', True, False, log2=13, bs=0.5, remove_fuzz=True, trim_df=False)
ans2 = uw.write(program2, 'script example 2', True, False, log2=10, remove_fuzz=True, trim_df=False)
ans3 = uw.write(program3, 'script example 3', True, False, log2=11, padding=2, remove_fuzz=True, trim_df=False)
# %timeit ans = uw.write(program, 'script example', False) #, False, log2=13, bs=0.5, remove_fuzz=True, trim_df=False)
ans = [ans1, ans2, ans3]

In [ ]:
for a in ans:
    a.report()
    a.plot('density', subplots=True, logy=True)

In [ ]:
ans[2].q(.999)

In [ ]:
uw['liaba']

In [ ]:
agg.Severity(**uw['liaba'][1]).plot()

In [ ]:
ans = uw.write('sev CA_PPA lognorm 12 cv 2; sev ilppa 12 * lognorm 2.1')
ans

In [ ]:
for a in ans:
    a.plot()

In [ ]:
portfolio_program = """
| name        | expos                 | limit                    | sev                                               | freq              |
|:------------|:----------------------|:-------------------------|:--------------------------------------------------|:------------------|
| big_mixture | 50 claims             | [50, 100, 150, 200] xs 0 | on lognorm 12 cv [1,2,3,4] wts [0.25 .25 .25 .25] | poisson           |
| A1a         | 500 premium at 0.5    |                          | on gamma 12 cv .30                                | mixed gamma 0.014 |
| A1b         | 500 premium at 0.5 lr |                          | on gamma 12 cv .30                                | mixed gamma 0.014 |
| A2          | 50  claims            | 30 xs 10                 | on gamma 12 cv .30                                | mixed gamma 0.014 |
| A3          | 50  claims            |                          | on gamma 12 cv .30                                | mixed gamma 0.014 |
| A4          | 50  claims            | 30 xs 20                 | on gamma 12 cv .30                                | mixed gamma 0.14  |
| hcmp        | 1e-8 * uw.cmp         |                          |                                                   |                   |
"""

In [ ]:
p = uw.write(portfolio_program, 'test_portfolio', update=True, verbose=False, log2=12, remove_fuzz=True)

In [ ]:
uw.portfolio['test_portfolio']

In [ ]:
p.recommend_bucket()

In [ ]:
p.bs

In [ ]:
p.report()  # subset of the audit report 

In [ ]:
port = p 
a = agg.axiter_factory(None, 24, aspect=1.4, height=2)
port.plot('quick', axiter=a)
port.plot('density', axiter=a, subplots=True, aspect=1.4, height=2)
port.plot('density', axiter=a, subplots=True, aspect=1.4, height=2, logy=True, ylim=[1e-10, 1e-2])
a.tidy()
agg.suptitle_and_tight('Density Plots for Portfolio 1')

In [ ]:
port.plot('audit', aspect=1.2, height=2.5)

In [ ]:
port.plot('priority', aspect=1.2, height=2.5)